<a href="https://colab.research.google.com/github/10100111/Diploma/blob/main/%D0%AD%D1%82%D0%B0%D0%BF_%E2%84%96_5_%D0%A1%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D1%84%D0%B8%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B9_%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D0%B8_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%BE%D0%B9_%D1%81%D0%B5%D1%82%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тема: «Прогнозирование маршрутов передвижения пассажиров Московского метрополитена на основании данных о валидации транспортных карт»

# Описание задачи:

- Предсказать, на какой станции и через какой промежуток времени, этот пассажир воспользуется метро повторно

- Описание столбцов для train и test:

● id - уникальный идентификатор столбца;

● ticket_id - уникальный идентификатор
билета, считается, что у одного
билета один владелец

● ticket_type_nm - тип билета

● entrance_id - уникальный id входа в станцию

● entrance_nm - название

● station_id - уникальное id станции захода

● station_nm - наименование станции захода

● line_id - уникальный id ветки на, которой находится станция

● line_nm - наименование ветки, на которой находится станция

● pass_dttm - дата валидации

● time_to_under (столбец для предсказания) - сколько времени прошло
между первой и второй валидацией

● label (столбец для предсказания) - id второй станции, на которой
произошла валидация




# База
По ссылке :  
https://drive.google.com/drive/folders/1c3nMb_moC6A9HfoX_6_Na-fy3JrDdRAe?usp=sharing

Вспомогательный файл к базе - subway.csv, весит 2 Гб.
Пока сохранено здесь https://drive.google.com/file/d/1gpPA9-JJe9zYDvr14mARJi8_uU1YGQmL/view?usp=sharing

# Параметризация данных

Поскольку основными способами решения задачи были выбраны autoML и алгоритмы классического ML, то требовалось предобработать датасет под оба варианта. Как в дальнейшем показал опыт, всегда требуется чистить данные, потому, что для дообучения лучших autoML моделей необработанные данные не подходят.

Исходные данные представляют собой DataFrame. В ходе обработки данных произведено:

 - Анализ данных. Пропусков не обнаружено. Основной поток пассажиров валидируется с интервалом 5-11 часов. Есть смысл разбить пассажиров на категории по типу проездного билета. В среднем один билет ездил 3,25 раза, следовательно стоит обратить внимание на создание признаков по ticket_id. Сенерированы признаки, коррелирующие с таргетами.

 - Визуализация данных.

 - После повторного анализа удалены избыточные признаки.  

[Ноутбук](https://colab.research.google.com/drive/1xr_19_YepUpNyrJ6_4ZbfnQPJtT_i9_p?usp=sharing) с визуализацией и деталями обработки датасета.


# Поиск модели
У нас 2 подзадачи:
 1. Предсказазние времени повторной валидации. Это задача регрессии.
 2. Предсказание станции повторной валидации. Это задача классификации на 276 классов (id станций).


В ходе подбора моделей также тестировались разные подходы к формированию датасета:
 - В формате DataFrame.
 - В формате numpy:
  - Преобразованные данные:
    - нормированные данные
    - данные переведенные в one hot encoding
  - Непреобразованные данные.

Для фиксации результатов использовался [журнал наблюдений](https://docs.google.com/spreadsheets/d/1gRSfXwXpUOmXeXONo83qpAdX-NFGnGQNd4PwqpYBh1g/edit?usp=sharing). В журнале сохранены ссылки на ноутбуки с эксприментами.

Поскольку autoMl считается самым быстрым способом валидации гипотез, подбор решения включал в себя несколько вариантов:

- Autokeras. Cразу было получено перспективное решение с моделью регрессии , метрика r2 = 0.83. Модель классификации дала ничтожно мало, recall = 0.05. Временные затраты на эксперименты заняли более 7 дней, так как ресурса Colab оказалось недостаточно. Приветствуется применение callback'ов для остановки обучения, если нет прогресса и можно сохранять лучшие веса, на случай, если обучение прервется. Будем считать решение для задачи предсказания времени второй валидации нашлось, а классификатор ищем дальше.

 - LGBM классификатор. Дал низкий показатель метрики recall = 0.06. Работает относительно быстро.

 - OPTUNA - библиотека для подбора гиперпараметров, использовалась для LGBM. Неоправдано ресурсоемкая задача, а результат слабый.

 - Autosklearn. Здесь автоподборщик AutoSklearnClassifier() выдал алгоритмическое решение на основе K-mean c cost = 0.99, но метрика recall  на данной модели оказалась самая худшая. Однако это меня навело на мысль использовать алгоритмы для решения задачи классификации.


Модель DecisionTreeClassifier() дал быстрый и высокий показатель метрики recall = 0.99. Предсказание номера станции второй валидации прошло успешно. Применение алгоритма деревьев решений, DecisionTreeRegressor(), в задаче предсказания времени второй валидации, дало также быстрый и наиболее точный результат r2 = 0.99.





# Результат дипломной работы



## Загрузка библиотек


In [ ]:
import pandas as pd

from sklearn.metrics import r2_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

from tensorflow.keras.models import load_model

from joblib import dump, load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Загрузка датасетов

In [ ]:
# Загружаем предобработанный датасет для задачи регрессии
x_train_reg = pd.read_csv('/content/drive/MyDrive/Базы/Metro/Datasets/datasets_df_clf/X_train_reg_df.csv')
x_test_reg = pd.read_csv('/content/drive/MyDrive/Базы/Metro/Datasets/datasets_df_clf/X_test_reg_df.csv')
y_train_reg = pd.read_csv('/content/drive/MyDrive/Базы/Metro/Datasets/datasets_df_clf/Y_train_reg_df.csv')
y_test_reg = pd.read_csv('/content/drive/MyDrive/Базы/Metro/Datasets/datasets_df_clf/Y_test_reg_df.csv')

x_train_reg.shape, x_test_reg.shape, y_train_reg.shape, y_test_reg.shape

((872816, 25), (218205, 25), (872816, 1), (218205, 1))

In [ ]:
# Загружаем предобработанный датасет для задачи классификации
X_train_clf = pd.read_csv('/content/drive/MyDrive/Базы/Metro/Datasets/datasets_df_clf/X_train_clf_df.csv')
X_test_clf = pd.read_csv('/content/drive/MyDrive/Базы/Metro/Datasets/datasets_df_clf/X_test_clf_df.csv')
y_train_clf = pd.read_csv('/content/drive/MyDrive/Базы/Metro/Datasets/datasets_df_clf/Y_train_clf_df.csv')
y_test_clf = pd.read_csv('/content/drive/MyDrive/Базы/Metro/Datasets/datasets_df_clf/Y_test_clf_df.csv')

X_train_clf.shape, X_test_clf.shape, y_train_clf.shape, y_test_clf.shape

((872816, 25), (218205, 25), (872816, 1), (218205, 1))

## Модели, score

In [ ]:
# Загружаем модель регрессии
reg = load('/content/drive/MyDrive/Базы/Metro/Models/DecisionTreeRegressor.joblib')

# Загружаем модель классификации
clf = load('/content/drive/MyDrive/Базы/Metro/Models/DecisionTreeClassifier_v2.joblib')

In [ ]:
# Предсказываем время следующей валидации на тестовом наборе данных
time_predict = reg.predict(x_test_reg)

# Предсказываем станцию валидации на тестовом наборе данных
station_predict = clf.predict(X_test_clf)

In [ ]:
# Вычисляем результат согласно формуле по ТЗ, где
# для классификации используем метрику recall,
# для регрессии испьзуем метрику r2
score = 0.5 * recall_score(y_test_clf, station_predict , average='macro') +  0.5 * r2_score(y_test_reg, time_predict)
score


0.9951046904175287

# Выводы

1. Неожиданностью явиловсь решение задачи с помощью модели алгоритма CART, а не модели нейронной сети.

2. Найденное решение появилось, когда пришлось изучить материалы по работе с несбалансированными классами, где CART показал себя особенно эффективно и по ресурсам и по времени, затраченному на реализацию.

3. AutoML оказался не таким уж быстрым и подходящим инструментом, относительно данной задачи.

4. В работе с моделями нейронных сетей всегда нужно использовать коллбеки для оптимизации поиска и сохранения результатов.

5. Всегда чистить данные насколько это возможно и преобразовывать в категории. При работе с нейронными сетями и autoML обязательно преобразовывать в OHE, иначе дообучение лучшей модели будет невозможно.

6. При работе с табличными данными в первую очередь применять классические алгоритмы, они могут дать быстрый и лучший результат.